In [ ]:
!pip install vllm peft transformers -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [ ]:

import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
anuragdeo_databricks_dolly_15k_path = kagglehub.dataset_download('anuragdeo/databricks-dolly-15k')
anuragdeo_llama3_2_3b_dolly_15k_qlora_4bit_pytorch_default_1_path = kagglehub.model_download('anuragdeo/llama3.2_3b_dolly_15k_qlora_4bit/PyTorch/default/1')

100%|██████████| 4.49M/4.49M [00:00<00:00, 47.4MB/s]

Extracting files...


100%|██████████| 882M/882M [00:11<00:00, 83.3MB/s]

Extracting files...


In [ ]:
print(anuragdeo_databricks_dolly_15k_path)
print(anuragdeo_llama3_2_3b_dolly_15k_qlora_4bit_pytorch_default_1_path)

/root/.cache/kagglehub/datasets/anuragdeo/databricks-dolly-15k/versions/1
/root/.cache/kagglehub/models/anuragdeo/llama3.2_3b_dolly_15k_qlora_4bit/PyTorch/default/1


In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-3B-Instruct-bnb-4bit")


# Load base 4-bit model
from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

base_model = AutoModelForCausalLM.from_pretrained(
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    quantization_config=bnb_config,
    device_map="auto",
)

# Merge LoRA
model = PeftModel.from_pretrained(base_model, "/root/.cache/kagglehub/models/anuragdeo/llama3.2_3b_dolly_15k_qlora_4bit/PyTorch/default/1/LLaMA-Factory/saves/dolly-llama-lora/")
model = model.merge_and_unload()

# Save merged model (convert to FP16 or BF16 if needed for vLLM)
save_path = "./merged_model_llama3"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:222: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


('./merged_model_llama3/tokenizer_config.json',
 './merged_model_llama3/special_tokens_map.json',
 './merged_model_llama3/chat_template.jinja',
 './merged_model_llama3/tokenizer.json')

In [ ]:
import json
with open("/root/.cache/kagglehub/datasets/anuragdeo/databricks-dolly-15k/versions/1/test_data.json") as f:
    dataset = json.load(f)

dataset[0]

{'instruction': 'Answer the given question, use context to answer whereever context is given',
 'input': 'Please classify each of the following as European or Asian countries: England, Thailand, Greece, Japan.',
 'label': 'England and Greece are European countries, while Thailand and Japan are Asian countries.'}

In [ ]:
from vllm import LLM, SamplingParams
import json

# Load tokenizer (for prompt formatting)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./merged_model_llama3")

# Format llama3 prompt
def format_llama3_prompt(instruction, input_text):
    user_prefix = "<|start_header_id|>user<|end_header_id|>\n"
    assistant_prefix = "<|start_header_id|>assistant<|end_header_id|>\n"
    eot = "<|eot_id|>"
    if input_text:
        return f"{user_prefix}{instruction}\n{input_text}{eot}\n{assistant_prefix}"
    else:
        return f"{user_prefix}{instruction}{eot}\n{assistant_prefix}"

# Load dataset
with open("/root/.cache/kagglehub/datasets/anuragdeo/databricks-dolly-15k/versions/1/test_data.json") as f:
    dataset = json.load(f)

prompts = [format_llama3_prompt(x["instruction"], x.get("input", "")) for x in dataset]

# Run inference with vLLM
llm = LLM(model="./merged_model_llama3", dtype="float16", max_model_len=8192)  # or "bfloat16"
sampling_params = SamplingParams(temperature=0.7, top_p=0.95, max_tokens=200)

outputs = llm.generate(prompts, sampling_params)

# Save results
results = []
for i, output in enumerate(outputs):
    prediction = output.outputs[0].text.strip()
    results.append({
        "instruction": dataset[i]["instruction"],
        "input": dataset[i].get("input", ""),
        "ground_truth": dataset[i]["label"],
        "prediction": prediction
    })

with open("vllm_inference_results.json", "w") as f:
    json.dump(results, f, indent=2)

print("✅ vLLM inference complete and saved.")


INFO 07-15 07:49:41 [__init__.py:244] Automatically detected platform cuda.
INFO 07-15 07:50:00 [config.py:841] This model supports multiple tasks: {'classify', 'embed', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 07-15 07:50:00 [config.py:1472] Using max model len 8192
WARNING 07-15 07:50:00 [config.py:960] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 07-15 07:50:00 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 07-15 07:50:01 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='./merged_model_llama3', speculative_config=None, tokenizer='./merged_model_llama3', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.BITSANDBYTES, tensor_parallel_size=1, pipeline_p

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-15 07:50:13 [model_runner.py:1203] Model loading took 2.2573 GiB and 9.825028 seconds
INFO 07-15 07:50:31 [worker.py:294] Memory profiling takes 17.53 seconds
INFO 07-15 07:50:31 [worker.py:294] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 07-15 07:50:31 [worker.py:294] model weights take 2.26GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.22GiB; the rest of the memory reserved for KV Cache is 9.75GiB.
INFO 07-15 07:50:32 [executor_base.py:113] # cuda blocks: 5702, # CPU blocks: 2340
INFO 07-15 07:50:32 [executor_base.py:118] Maximum concurrency for 8192 tokens per request: 11.14x
INFO 07-15 07:50:36 [model_runner.py:1513] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decre

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 07-15 07:52:36 [model_runner.py:1671] Graph capturing finished in 120 secs, took 0.60 GiB
INFO 07-15 07:52:36 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 143.10 seconds


Adding requests:   0%|          | 0/3003 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3003 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

✅ vLLM inference complete and saved.


In [ ]:
# prompt: Download the file ./vllm_inference_results.json from colab

from google.colab import files
files.download('vllm_inference_results.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# prompt: Use the evaluate library and calculate the all the rouge and bleu scores for the elements in results

# !pip install evaluate sacrebleu rouge_score -q
import evaluate

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

predictions = [item["prediction"] for item in results]
references = [[item["ground_truth"]] for item in results] # Sacrebleu expects a list of lists for references

rouge_scores = rouge.compute(predictions=predictions, references=references)
bleu_score = [bleu.compute(predictions=predictions, references=references, max_order=1),bleu.compute(predictions=predictions, references=references, max_order=2),bleu.compute(predictions=predictions, references=references, max_order=3),bleu.compute(predictions=predictions, references=references)]

print("ROUGE Scores:", rouge_scores)
print("BLEU Score:", bleu_score)

ROUGE Scores: {'rouge1': np.float64(0.3811193257552386), 'rouge2': np.float64(0.19473645991168875), 'rougeL': np.float64(0.3007642309423968), 'rougeLsum': np.float64(0.3254178882502138)}
BLEU Score: [{'bleu': 0.2774216881994816, 'precisions': [0.2774216881994816], 'brevity_penalty': 1.0, 'length_ratio': 1.2175782989279615, 'translation_length': 248845, 'reference_length': 204377}, {'bleu': 0.1664269610830033, 'precisions': [0.2774216881994816, 0.0998405479942402], 'brevity_penalty': 1.0, 'length_ratio': 1.2175782989279615, 'translation_length': 248845, 'reference_length': 204377}, {'bleu': 0.11256896347918528, 'precisions': [0.2774216881994816, 0.0998405479942402, 0.05150016675381784], 'brevity_penalty': 1.0, 'length_ratio': 1.2175782989279615, 'translation_length': 248845, 'reference_length': 204377}, {'bleu': 0.08263376352755085, 'precisions': [0.2774216881994816, 0.0998405479942402, 0.05150016675381784, 0.032686929812497136], 'brevity_penalty': 1.0, 'length_ratio': 1.217578298927961

In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [2]:
# prompt: Load the results from the json file and then take the predictions and reference from it
import json
with open("qlora_inference_results.json", "r") as f:
    loaded_results = json.load(f)

predictions = [item["prediction"] for item in loaded_results]
references = [[item["ground_truth"]] for item in loaded_results] # Sacrebleu expects a list of lists for references

In [3]:
# prompt: Write a code to get the bert score also
import evaluate
!pip install bert-score -q
bertscore = evaluate.load("bertscore")

bert_scores = bertscore.compute(
    predictions=predictions,
    references=references,
    lang="en",
    device="cuda"  # ✅ This makes it use the GPU
)

print("BERT Score:", bert_scores)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.0 MB/s eta 0:00:00


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT Score: {'precision': [0.8233639001846313, 0.8521376848220825, 0.8078973889350891, 0.8815902471542358, 0.8675154447555542, 0.8411484956741333, 0.8570541143417358, 0.8712025284767151, 0.9074949622154236, 0.8276589512825012, 0.8542828559875488, 0.8737449049949646, 0.9218403100967407, 0.8403904438018799, 0.8223563432693481, 0.8175188899040222, 0.8800279498100281, 0.9486493468284607, 0.9112372398376465, 0.8774186372756958, 0.8693089485168457, 0.877213716506958, 0.856089174747467, 0.9814786911010742, 0.8398964405059814, 0.8338207006454468, 0.8711145520210266, 0.9613122344017029, 0.8478976488113403, 0.8252800703048706, 0.8180066347122192, 0.8411633372306824, 0.7937891483306885, 0.908623456954956, 0.9362295866012573, 0.8500080108642578, 0.8317916393280029, 0.8823865652084351, 0.9734293222427368, 0.992364227771759, 0.8471579551696777, 0.8740243911743164, 0.8507957458496094, 0.9141483902931213, 0.9097409248352051, 0.8280290365219116, 0.8100961446762085, 0.8488442897796631, 0.852151036262512

In [4]:
# prompt: Calculate the average precision, recall and f1 from bert_scores dictonary

# Calculate average precision, recall, and f1
average_precision = sum(bert_scores['precision']) / len(bert_scores['precision'])
average_recall = sum(bert_scores['recall']) / len(bert_scores['recall'])
average_f1 = sum(bert_scores['f1']) / len(bert_scores['f1'])

print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1 Score: {average_f1}")


Average Precision: 0.8696942146404799
Average Recall: 0.8747995734770537
Average F1 Score: 0.8716116527140716
